In [1]:
from glob import glob


import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

from sklearn.svm import OneClassSVM

In [2]:
path = glob('../../data/raw_data/*.xlsx')
path

['../../data/raw_data\\scenario_1.xlsx',
 '../../data/raw_data\\scenario_10.xlsx',
 '../../data/raw_data\\scenario_11.xlsx',
 '../../data/raw_data\\scenario_12.xlsx',
 '../../data/raw_data\\scenario_13.xlsx',
 '../../data/raw_data\\scenario_14.xlsx',
 '../../data/raw_data\\scenario_2.xlsx',
 '../../data/raw_data\\scenario_3.xlsx',
 '../../data/raw_data\\scenario_4.xlsx',
 '../../data/raw_data\\scenario_5.xlsx',
 '../../data/raw_data\\scenario_6.xlsx',
 '../../data/raw_data\\scenario_7.xlsx',
 '../../data/raw_data\\scenario_8.xlsx',
 '../../data/raw_data\\scenario_9.xlsx']

In [3]:
scenarios = [i.split('\\')[-1].split('.xlsx')[0] for i in path]
scenarios

['scenario_1',
 'scenario_10',
 'scenario_11',
 'scenario_12',
 'scenario_13',
 'scenario_14',
 'scenario_2',
 'scenario_3',
 'scenario_4',
 'scenario_5',
 'scenario_6',
 'scenario_7',
 'scenario_8',
 'scenario_9']

loading data

In [4]:
df = pd.read_excel(path[0],skiprows =1, keep_default_na = True,nrows=1850)
df.dropna(axis=1,inplace = True)
df.columns = ['Time', 'pipeline_1', 'pipeline_2', 'pipeline_3', 'sep_lvl']
df.head()

,Time,pipeline_1,pipeline_2,pipeline_3,sep_lvl
0,0.000000,11.138935,11.085769,11.484986,0.476073
1,0.100000,11.138756,11.085440,11.484453,0.475311
2,0.233333,11.138499,11.085223,11.484049,0.474378
3,0.411111,11.138120,11.084999,11.483658,0.473195
4,0.648148,11.137534,11.084570,11.483118,0.471706


1. Определяем min/max values для столбцов

In [5]:
for i in df.columns[1:]:
    a = df.loc[:,i]
    if (a.max() - a.min()) < 0.2:
        a = a.mean()

    df.loc[:,i] = a

2. sep_lvl window 100

In [6]:
# сглаживаем
WINDOW_SIZE = 100

df['sep_lvl'] = df.sep_lvl.rolling(WINDOW_SIZE).mean()

3. Ищем рост давления:

In [7]:
# сначала нужно сделать дроп NA. чтобы не было путаницы после предикта
df.dropna(inplace=True)

In [9]:
# давление
START_POS = 0
a = np.array(df.pipeline_3)
oc_svm = OneClassSVM(nu=.001, kernel="rbf", gamma='scale')
oc_svm.fit(a[START_POS:].reshape(-1,1))
pred = oc_svm.predict(a[START_POS:].reshape(-1,1))
if np.argmin(pred) == 0 : print('No slug') # То значит этот трубопровод НЕ участвует в пробке 
else: 
    pressure_start_raising = np.argmin(pred)

4. Ищем рост пробки:

In [10]:
# тут особенность, применяем скользящее окно еще раз - тогда диспесия и средняя во время роста явно будет отличаться:
a = np.array(df.sep_lvl.rolling(100).mean()) 

oc_svm = OneClassSVM(nu=.24, kernel="sigmoid", gamma='scale')  # именно сигмоидное ядро
oc_svm.fit(a[300:].reshape(-1,1)) # 100 - это скользящее окно + делаем большой запас 200, так как в начале сценариев (о-уммолчанию)сепаратор полный

pred = oc_svm.predict(a[300:].reshape(-1,1))

liquid_start_raising = np.argmax(pred)+300

Ищем дельту = разницу между началом роста давления и началом роста жидкости в сепараторе.

In [11]:
delta = liquid_start_raising - pressure_start_raising

Теперь target нужно брать с delta : df.sep_lvl[255:]

In [12]:
y = df.sep_lvl[255:]

Делаем lagij для всех трубопроводов

In [13]:
from copy import copy

In [14]:
df_copied = copy(df)
df_copied.head()

,Time,pipeline_1,pipeline_2,pipeline_3,sep_lvl
99,19.750891,11.067844,11.040846,11.463766,0.345969
100,20.291411,11.067844,11.040846,11.463157,0.343197
101,21.012105,11.067844,11.040846,11.462391,0.340144
102,21.068150,11.067844,11.040846,11.462319,0.337113
103,21.106283,11.067844,11.040846,11.462264,0.334095


In [15]:
for i in df_copied.columns[1:-1]:
    for j in range(1,301):
        df_copied[f'{i}_delta_with_lag_{j}'] = df_copied[i] - df_copied[i].shift(j)


In [16]:
df_copied.dropna(inplace=True)

y = np.array(df_copied.sep_lvl)[delta:] 
df_copied.drop(columns=df_copied.columns[:5], inplace=True)

Сохраняемся:  
X (features - dataframe) -> parquet  
y (target - numpy) -> binary

In [18]:
df_copied.to_parquet(f'../../data/prepared/{scenarios[0]}.parquet')

y.tofile(f'../../data/prepared/{scenarios[0]}.bite')